Rule mining of overdose data. Importing libraries needed to clean data and prepare for Apiori algorithm. 

In [28]:
import numpy as np
import pandas as pd

Loading necessary data into a dataframe and printing first 5 rows

In [29]:
df = pd.read_csv('C:\Accidental-Overdose-Data.csv',usecols=['_id','death_date_and_time','combined_od1','combined_od2','combined_od3','combined_od4',
                                                        'combined_od5','combined_od6','combined_od7','combined_od8','combined_od9','combined_od10'])
df.head()

,_id,death_date_and_time,combined_od1,combined_od2,combined_od3,combined_od4,combined_od5,combined_od6,combined_od7,combined_od8,combined_od9,combined_od10
0,8273327,2007-01-30T09:22:00,Amitriptyline,Demoxepam,Fluoxetine,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8273328,2007-01-26T14:25:00,Alprazolam,Doxepin,Oxycodone,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8273329,2007-01-31T06:01:00,Cocaine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8273330,2007-03-05T13:32:00,Cocaine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8273331,2007-03-09T22:22:00,Fentanyl,Hydrocodone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Handling null values and joining all columns for each basket itemset

In [30]:
df = df.fillna('')
df['DrugsinSystem'] = df[['combined_od1','combined_od2','combined_od3','combined_od4','combined_od5','combined_od6','combined_od7','combined_od8',
                          'combined_od9','combined_od10']].apply(lambda x: ','.join(filter(None,x)),axis=1)
df.head()

,_id,death_date_and_time,combined_od1,combined_od2,combined_od3,combined_od4,combined_od5,combined_od6,combined_od7,combined_od8,combined_od9,combined_od10,DrugsinSystem
0,8273327,2007-01-30T09:22:00,Amitriptyline,Demoxepam,Fluoxetine,,,,,,,,"Amitriptyline,Demoxepam,Fluoxetine"
1,8273328,2007-01-26T14:25:00,Alprazolam,Doxepin,Oxycodone,,,,,,,,"Alprazolam,Doxepin,Oxycodone"
2,8273329,2007-01-31T06:01:00,Cocaine,,,,,,,,,,Cocaine
3,8273330,2007-03-05T13:32:00,Cocaine,,,,,,,,,,Cocaine
4,8273331,2007-03-09T22:22:00,Fentanyl,Hydrocodone,,,,,,,,,"Fentanyl,Hydrocodone"


Dropping uncessary columns and printing information on the dataset

In [31]:
df.drop(columns =['death_date_and_time','combined_od1','combined_od2','combined_od3','combined_od4','combined_od5','combined_od6','combined_od7','combined_od8','combined_od9','combined_od10'], inplace = True)
df.head()

,_id,DrugsinSystem
0,8273327,"Amitriptyline,Demoxepam,Fluoxetine"
1,8273328,"Alprazolam,Doxepin,Oxycodone"
2,8273329,Cocaine
3,8273330,Cocaine
4,8273331,"Fentanyl,Hydrocodone"


In [32]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7024 entries, 0 to 7023
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   _id            7024 non-null   int64 
 1   DrugsinSystem  7024 non-null   object
dtypes: int64(1), object(1)
memory usage: 109.9+ KB


_id              0
DrugsinSystem    0
dtype: int64

In [33]:
df.drop_duplicates()

,_id,DrugsinSystem
0,8273327,"Amitriptyline,Demoxepam,Fluoxetine"
1,8273328,"Alprazolam,Doxepin,Oxycodone"
2,8273329,Cocaine
3,8273330,Cocaine
4,8273331,"Fentanyl,Hydrocodone"
...,...,...
7019,8280346,"Fentanyl,Alcohol,Isopropanol"
7020,8280347,"Fentanyl,Acetyl Fentanyl,Cocaine,Despropionyl ..."
7021,8280348,"Alcohol,Cocaine,Fentanyl"
7022,8280349,"Fentanyl,Acetyl Fentanyl,Methamphetamine,Cocaine"


Splitting joined itemsets into lists 

In [36]:
df['DrugList'] = df['DrugsinSystem'].str.split(',')
df.head()

,_id,DrugsinSystem,DrugList
0,8273327,"Amitriptyline,Demoxepam,Fluoxetine","[Amitriptyline, Demoxepam, Fluoxetine]"
1,8273328,"Alprazolam,Doxepin,Oxycodone","[Alprazolam, Doxepin, Oxycodone]"
2,8273329,Cocaine,[Cocaine]
3,8273330,Cocaine,[Cocaine]
4,8273331,"Fentanyl,Hydrocodone","[Fentanyl, Hydrocodone]"


Importing necessary library for Apiori algorithm

In [38]:
!pip install mlxtend
import mlxtend
from mlxtend.preprocessing import TransactionEncoder

Encoding basket's itemsets into binary values

In [43]:
encoder = TransactionEncoder()
array = encoder.fit(df['DrugList']).transform(df['DrugList'])
df_encoded = pd.DataFrame(array, columns = encoder.columns_)
df_encoded.head(50)

,1,1-Difluoroethane,2-Fluoro Deschloroketamine,3-Dimethyl-Butanoic Acid,3-Methylfentanyl,4-Methoxy-Butyryl Fentanyl,4-fluoro-MDMB-BINACA,5F-ADB 3,7-Aminoclonazepam,Acetaminophen,...,U-48800 Synthetic Opioid,Valeryl Fentanyl,Valproic Acid,Venlafaxine,Verapamil,Xylazine,Zolpidem,Zopiclone,n-desethyl isotonitazene,p-Fluoroisobutyrylfentanyl/m-FIBF
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Importing libaries for frequent itemsets and assciation rules and finding frequent items.

In [41]:
from mlxtend.frequent_patterns import apriori, association_rules

In [49]:
frequent_itemsets = apriori(df_encoded, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.094391,(Acetyl Fentanyl),1
1,0.230211,(Alcohol),1
2,0.105068,(Alprazolam),1
3,0.414721,(Cocaine),1
4,0.559083,(Fentanyl),1
5,0.363895,(Heroin),1
6,0.074886,(Methadone),1
7,0.053531,(Morphine),1
8,0.086418,(Oxycodone),1
9,0.084425,(Para-Fluorofentanyl),1


In [51]:
frequent_itemsets.sort_values(['support'], ascending=False)

,support,itemsets,length
4,0.559083,(Fentanyl),1
3,0.414721,(Cocaine),1
5,0.363895,(Heroin),1
14,0.257973,"(Fentanyl, Cocaine)",2
1,0.230211,(Alcohol),1
16,0.207005,"(Fentanyl, Heroin)",2
15,0.127278,"(Cocaine, Heroin)",2
12,0.117312,"(Fentanyl, Alcohol)",2
2,0.105068,(Alprazolam),1
11,0.097523,"(Cocaine, Alcohol)",2


Finding association rules of confidence, lift, and support 

In [57]:
rules = association_rules(frequent_itemsets,metric = "confidence", min_threshold=0.60)
rules = rules.sort_values(['confidence','lift'], ascending = [False, False])
rules.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(Para-Fluorofentanyl),(Fentanyl),0.084425,0.559083,0.083713,0.991568,1.773561,0.036512,52.292739,0.476381
0,(Acetyl Fentanyl),(Fentanyl),0.094391,0.559083,0.092682,0.981900,1.756269,0.039910,24.360656,0.475493
4,"(Cocaine, Heroin)",(Fentanyl),0.127278,0.559083,0.085706,0.673378,1.204433,0.014547,1.349930,0.194488
3,"(Cocaine, Alcohol)",(Fentanyl),0.097523,0.559083,0.060792,0.623358,1.114964,0.006268,1.170651,0.114252
1,(Cocaine),(Fentanyl),0.414721,0.559083,0.257973,0.622039,1.112606,0.026109,1.166567,0.172924


In [65]:
rules = association_rules(frequent_itemsets,metric = "lift", min_threshold=1)
rules = rules.sort_values(['lift','confidence'], ascending = [False, False])
rules.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
8,(Para-Fluorofentanyl),(Fentanyl),0.084425,0.559083,0.083713,0.991568,1.773561,0.036512,52.292739,0.476381
9,(Fentanyl),(Para-Fluorofentanyl),0.559083,0.084425,0.083713,0.149733,1.773561,0.036512,1.076809,0.989218
0,(Fentanyl),(Acetyl Fentanyl),0.559083,0.094391,0.092682,0.165775,1.756269,0.039910,1.085570,0.976627
1,(Acetyl Fentanyl),(Fentanyl),0.094391,0.559083,0.092682,0.981900,1.756269,0.039910,24.360656,0.475493
11,"(Fentanyl, Alcohol)",(Cocaine),0.117312,0.414721,0.060792,0.518204,1.249524,0.012140,1.214786,0.226236
14,(Cocaine),"(Fentanyl, Alcohol)",0.414721,0.117312,0.060792,0.146584,1.249524,0.012140,1.034300,0.341197
16,"(Cocaine, Heroin)",(Fentanyl),0.127278,0.559083,0.085706,0.673378,1.204433,0.014547,1.349930,0.194488
17,(Fentanyl),"(Cocaine, Heroin)",0.559083,0.127278,0.085706,0.153298,1.204433,0.014547,1.030731,0.384956
12,"(Cocaine, Alcohol)",(Fentanyl),0.097523,0.559083,0.060792,0.623358,1.114964,0.006268,1.170651,0.114252
13,(Fentanyl),"(Cocaine, Alcohol)",0.559083,0.097523,0.060792,0.108734,1.114964,0.006268,1.012579,0.233854


In [64]:
rules = association_rules(frequent_itemsets,metric = "support", min_threshold=0.05)
rules = rules.sort_values(['support','confidence'], ascending = [False, False])
rules.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
9,(Cocaine),(Fentanyl),0.414721,0.559083,0.257973,0.622039,1.112606,0.026109,1.166567,0.172924
8,(Fentanyl),(Cocaine),0.559083,0.414721,0.257973,0.461421,1.112606,0.026109,1.086710,0.229542
13,(Heroin),(Fentanyl),0.363895,0.559083,0.207005,0.568858,1.017483,0.003557,1.022671,0.027012
12,(Fentanyl),(Heroin),0.559083,0.363895,0.207005,0.370257,1.017483,0.003557,1.010103,0.038970
11,(Heroin),(Cocaine),0.363895,0.414721,0.127278,0.349765,0.843375,-0.023637,0.900104,-0.225978
10,(Cocaine),(Heroin),0.414721,0.363895,0.127278,0.306900,0.843375,-0.023637,0.917768,-0.240875
5,(Alcohol),(Fentanyl),0.230211,0.559083,0.117312,0.509586,0.911467,-0.011395,0.899070,-0.112043
4,(Fentanyl),(Alcohol),0.559083,0.230211,0.117312,0.209829,0.911467,-0.011395,0.974206,-0.180528
3,(Alcohol),(Cocaine),0.230211,0.414721,0.097523,0.423624,1.021468,0.002050,1.015447,0.027301
2,(Cocaine),(Alcohol),0.414721,0.230211,0.097523,0.235153,1.021468,0.002050,1.006461,0.035908
